
## Trial to combine middle ear and bruxism tagging to identify pure moments of middle ear activation

In [7]:
import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')
import mne
from tinnsleep.config import Config
from tinnsleep.create_reports import preprocess, reporting
from tinnsleep.data import CreateRaw, RawToEpochs_sliding, CleanAnnotations, AnnotateRaw_sliding
from tinnsleep.classification import AmplitudeThresholding
from tinnsleep.check_impedance import create_annotation_mne, Impedance_thresholding_sliding, check_RMS, fuse_with_classif_result
from tinnsleep.signal import rms
from tinnsleep.scoring import classif_to_burst, burst_to_episode, create_list_events
import warnings
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import pandas as pd
import pickle
import scipy
from tinnsleep.config import Config

print("Config loaded")


Config loaded


In [8]:
#List of MEMA files, hardcoded, to be modified
print(Config.bruxisme_files[44:46])
print("")
ME_files=[Config.bruxisme_files[5], Config.bruxisme_files[9], 
          Config.bruxisme_files[14], Config.bruxisme_files[22]]
ME_files.extend(Config.bruxisme_files[28:44])
ME_files.extend(Config.bruxisme_files[46:])

print(ME_files)
print(len(ME_files))

['E:/Acou_sommeil/EDF_V2_PAUL\\robin_nuit_23_sept.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\robin_nuit_son_24_sept.edf']

['E:/Acou_sommeil/EDF_V2_PAUL\\1DA15_nuit_son.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1GB19_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1MF19_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1RA17_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\HZB_nuit_1.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\HZB_nuit_2.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\HZB_nuit_3.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_1_dec_OD__0to0.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_3_dec_OD__4to0to2.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_4_dec_OD__3to3.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_5_dec_OD__0to1.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHM_nuit_1.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHM_nuit_2.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHM_nuit_3.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\Schmidtlin_nuit_2_dec_3to0to4.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\Unger_2.edf', 'E:/Acou_somm

## Setting down parameters for bruxism detection

In [16]:
#Setting parameters
os.chdir("C:/Users/Zeta/Documents/acou_sommeil_HD_ENS/Tinnitus-n-Sleep/Notebooks")
THR_classif=[[0,2]]
sfreq = 250
window_length_BRUX = 0.25                    # in seconds
duration_BRUX = int(window_length_BRUX * sfreq)   # in samples
interval_BRUX = duration_BRUX                     # no overlapping
n_adaptive = 480 # number of epochs for adaptative baseline

window_length_MEMA = 1                    # in seconds
duration_MEMA = int(window_length_MEMA * sfreq)   # in samples
interval_MEMA = duration_MEMA                     # no overlapping

#Importing personnalized parameters for dataset
df = pd.read_pickle("data/valid_chans_THR_imp.pk")
dico_chans= df.to_dict("list")

#IMPORTANT:
max_duration_between_bursts = 1.5  #maximal duration allowed between MEMA bursts to decide or not to join them together


print("parameters set")

parameters set


## Bruxism + MEMA processing for pure MEMA visualisation

In [19]:
filenames = ME_files  # load file from config
#filenames=['E:/Acou_sommeil/EDF_V2_PAUL\\sophie_mema.edf']

#Output of the processing, stock pure MEMA analysis output
ME_reports={}


with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    print("Files processed : ")
    
    #Loop on all the patient files
    for filename in filenames:
        
            print(filename.split("\\")[-1])
            #-----------------MEMA processing preparation ---------------------------------------
            raw = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading
            picks_chan = ['Airflow']           # middle ear electrode
            raw  = CreateRaw(raw[picks_chan][0], raw.info["sfreq"], picks_chan, ch_types=['emg']) # pick channels and load
            ch_names = raw.info["ch_names"]
            print("Data filtered")

            #epoching
            epochs = RawToEpochs_sliding(raw, duration=duration_MEMA, interval=interval_MEMA)
            print(f"Epochs done, shape {epochs.shape}")

            #-----------------MEMA processing Foward-backward ---------------------------------------
            #Foward
            # compute the sum of power over electrodes and samples in each window
            pipeline = AmplitudeThresholding(abs_threshold=0., rel_threshold=3.5, n_adaptive=60)
            X        = rms(epochs) # take only valid labels
            labels_f   = pipeline.fit_predict(X)


            #Backward
            #Reversing epochs array
            epochs = epochs[::-1]
             # compute the sum of power over electrodes and samples in each window
            pipeline = AmplitudeThresholding(abs_threshold=0., rel_threshold=3.5, n_adaptive=60)
            X        = rms(epochs) # take only valid labels
            labels   = pipeline.fit_predict(X)
            #Reversing labels
            labels_b = labels[::-1]


            #-----------------MEMA foward-backward merge ---------------------------------------
            # Logical OR -- merged backward and foward
            labels_fb = np.any(np.c_[labels_f, labels_b], axis=-1)


            #-----------------MEMA / bruxism merge preparation---------------------------------------
            #adaptation of labels_fb from 1s epochs to 0,25s epochs
            labels_fb_shep=[]  
            for elm in labels_fb:
                for i in range(4):
                    labels_fb_shep.append(elm)


            #-----------------MEMA bursts conversion to episodes ----------------------------------------
            OM_burst = classif_to_burst(labels_fb_shep, time_interval=window_length_BRUX)
            OM_ep= burst_to_episode(OM_burst, delim=max_duration_between_bursts, min_burst_joining=0)
            li_OM = create_list_events(OM_ep, window_length_BRUX, window_length_BRUX* len(labels_fb_shep))

            #All episodes as tonic
            for elm in li_OM:
                if elm!=0:
                    elm=True
                else:
                    elm = False

            #Should work without these lines since episode fix with min_burst_joining=0
            miny = min(len(li_OM), len(labels_fb_shep))
            MEMA = np.any(np.c_[li_OM[:miny], labels_fb_shep[:miny]], axis=-1)



            print("number of middle ear events")
            print(len(OM_ep))



            

Files processed : 
1DA15_nuit_son.edf
Data filtered
Epochs done, shape (32460, 1, 200)
number of middle ear events
43
1GB19_nuit_hab.edf
Data filtered
Epochs done, shape (31810, 1, 200)
number of middle ear events
338
1MF19_nuit_hab.edf
Data filtered
Epochs done, shape (32560, 1, 200)
number of middle ear events
104
1RA17_nuit_hab.edf
Data filtered
Epochs done, shape (32837, 1, 200)
number of middle ear events
293
HZB_nuit_1.edf
Data filtered
Epochs done, shape (24520, 1, 200)
number of middle ear events
158
HZB_nuit_2.edf
Data filtered
Epochs done, shape (25080, 1, 200)
number of middle ear events
157
HZB_nuit_3.edf
Data filtered
Epochs done, shape (24970, 1, 200)
number of middle ear events
92
SCHMIDTLIN_nuit_1_dec_OD__0to0.edf
Data filtered
Epochs done, shape (31010, 1, 200)
number of middle ear events
196
SCHMIDTLIN_nuit_3_dec_OD__4to0to2.edf
Data filtered
Epochs done, shape (35380, 1, 200)
number of middle ear events
279
SCHMIDTLIN_nuit_4_dec_OD__3to3.edf
Data filtered
Epochs done

In [11]:
for elm in ME_reports.keys():
    print(elm)
    print(ME_reports[elm][1])

E:/Acou_sommeil/EDF_V2_PAUL\1DA15_nuit_son.edf
15
E:/Acou_sommeil/EDF_V2_PAUL\1GB19_nuit_hab.edf
141
E:/Acou_sommeil/EDF_V2_PAUL\1MF19_nuit_hab.edf
37
E:/Acou_sommeil/EDF_V2_PAUL\HZB_nuit_1.edf
70
E:/Acou_sommeil/EDF_V2_PAUL\HZB_nuit_2.edf
49
E:/Acou_sommeil/EDF_V2_PAUL\HZB_nuit_3.edf
22
E:/Acou_sommeil/EDF_V2_PAUL\SCHMIDTLIN_nuit_3_dec_OD__4to0to2.edf
142
E:/Acou_sommeil/EDF_V2_PAUL\SCHMIDTLIN_nuit_4_dec_OD__3to3.edf
110
E:/Acou_sommeil/EDF_V2_PAUL\SCHMIDTLIN_nuit_5_dec_OD__0to1.edf
101
E:/Acou_sommeil/EDF_V2_PAUL\SCHM_nuit_1.edf
6
E:/Acou_sommeil/EDF_V2_PAUL\SCHM_nuit_2.edf
87
E:/Acou_sommeil/EDF_V2_PAUL\SCHM_nuit_3.edf
147
E:/Acou_sommeil/EDF_V2_PAUL\Schmidtlin_nuit_2_dec_3to0to4.edf
0
E:/Acou_sommeil/EDF_V2_PAUL\Unger_2.edf
272
E:/Acou_sommeil/EDF_V2_PAUL\jon_mema.edf
182
E:/Acou_sommeil/EDF_V2_PAUL\jose_mema.edf
29
E:/Acou_sommeil/EDF_V2_PAUL\robin_mema_nuit_1.edf
78
E:/Acou_sommeil/EDF_V2_PAUL\robin_mema_nuit_2.edf
166
E:/Acou_sommeil/EDF_V2_PAUL\sophie_mema.edf
130
E:/Acou_somme

## Small function to check if all channel selections are good

In [5]:
import warnings

labels_subj={}
EDF_list = Config.bruxisme_files

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    results={}
    print("Files processed : ")
    for filename in EDF_list:
        #opens the raw file
        raw = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading
        print(filename.split("\\")[-1])
        #Get channels indexes
        ind_picks_chan= dico_chans[filename.split("\\")[-1]][0]
        ind_picks_imp= dico_chans[filename.split("\\")[-1]][1]
        #Get THR_imp value for filename
        THR_imp = dico_chans[filename.split("\\")[-1]][2]
        #print(raw.info["ch_names"])
        #Get channel names from indexes
        if len(ind_picks_chan)>0: #ignore file if no channel is good
            picks_chan=[]
            for elm in ind_picks_chan:
                picks_chan.append(raw.info["ch_names"][elm])
            picks_imp=[]
            for elm in ind_picks_imp:
                picks_imp.append(raw.info["ch_names"][elm])
            print(picks_chan)
            print(picks_imp)
            print("")

Files processed : 
1BA07_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1BA07_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1CC05_nuit_hab.edf
['1', '2']
['1 Impedance', '2 Impedance']

1CC05_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1DA15_nuit_hab.edf
['1']
['1 Imp?dance']

1DA15_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1DL12_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1DL12_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1GB18_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1GB19_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1GF14_nuit_hab.edf
1GF14_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1MA16_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1MA16_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1MF19_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1MF19_nuit_son.edf
['1', '2']
['1 Impedance', '2 Impedance']

1MN09_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp